# Imports & setup

In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
import pandas as pd
import numpy as np
import pickle
from xgboost import XGBRegressor
from PositionalModel import PositionalModel

from src.match_names import *
from config import *

In [3]:
info = ["Name_original", "GW", "Season", "Team", "Opponent", "Was_home", "Team_rating", "Opp_rating", "FPL_pos", "Price"]
to_predict = ["FPL_points"]

In [4]:
len(FEATURES_OUTFIELD)

42

# Reading from files

In [5]:
df = pd.read_csv("data/upcoming/upcoming_fixtures.csv")
CURRENT_SEASON = "2023-24"

C:\users\user\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (2,3,6,8,36,43,44,45,59,84,86,147,208) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
df.shape

(34514, 274)

In [7]:
df.columns

Index(['Unnamed: 0', 'Date', 'Day', 'Venue', 'Team', 'Opponent', 'Name',
       'Start', 'Pos', 'Min',
       ...
       'FPL_xGC_30', 'Opp_rating_30', 'Team_rating_30', 'Min_points_30',
       'Team_result_30', 'DEF', 'FWD', 'GK', 'MID', 'Finished'],
      dtype='object', length=274)

In [8]:
# df = df[df["GW"] == 2]

In [9]:
# X_test = df[df["Season"] == CURRENT_SEASON][FEATURES_OUTFIELD]
df = df[df["Season"] == CURRENT_SEASON]
# .reset_index(drop=True)

In [10]:
df.isna().sum().sort_values(ascending=False).head(20)

FPL_GW           16744
Min_points_30    16744
Clean_name       16744
Min_points_4     16744
Min_points_2     16744
Min_points       16744
Cmp%             15960
npxG             15954
SCA              15954
GCA              15954
Cmp              15954
Att              15954
Carries          15954
PrgP             15954
Blocks           15954
PrgC             15954
T_att            15954
T_succ           15954
xG               15954
xA               15954
dtype: int64

In [11]:
# df[FEATURES_OUTFIELD].iloc[0]

In [12]:
df = df[~df[FEATURES_OUTFIELD].isnull().any(axis=1)] # for some reason 45 rows are missing some feature values

In [13]:
df.shape

(16744, 274)

In [14]:
X_test = df.copy()

In [15]:
df.columns

Index(['Unnamed: 0', 'Date', 'Day', 'Venue', 'Team', 'Opponent', 'Name',
       'Start', 'Pos', 'Min',
       ...
       'FPL_xGC_30', 'Opp_rating_30', 'Team_rating_30', 'Min_points_30',
       'Team_result_30', 'DEF', 'FWD', 'GK', 'MID', 'Finished'],
      dtype='object', length=274)

In [16]:
file = open("models/GBR_positional.pkl",'rb')
model = pickle.load(file)

In [17]:
model

In [18]:
model.predict( X_test.head(5) )

[1.5853266289830468,
 1.2726693622326923,
 2.0793286384067575,
 1.9488243331970216,
 2.1122465307299487]

In [19]:
model.predict( X_test.head(3) )

[1.5853266289830468, 1.2726693622326923, 2.0793286384067575]

In [20]:
model.predict( X_test.head(4) )

[1.5853266289830468,
 1.2726693622326923,
 2.0793286384067575,
 1.9488243331970216]

In [21]:
X_test.head(5)

,Unnamed: 0,Date,Day,Venue,Team,Opponent,Name,Start,Pos,Min,...,FPL_xGC_30,Opp_rating_30,Team_rating_30,Min_points_30,Team_result_30,DEF,FWD,GK,MID,Finished
76,76,2023-08-20,Sun,Home,West Ham,Chelsea,Aaron-Cresswell,0.0,0,0.0,...,0.00,1777.369108,1761.774133,NaN,1.166667,1.0,0.0,0.0,0.0,True
77,77,2023-08-26,Sat,Away,West Ham,Brighton,Aaron-Cresswell,0.0,0,0.0,...,0.00,1780.288375,1762.710238,NaN,1.166667,1.0,0.0,0.0,0.0,True
106,106,2023-08-13,Sun,Home,Brentford,Tottenham,Aaron-Hickey,1.0,RB,71.0,...,1.02,1773.661743,1765.611925,NaN,1.428571,1.0,0.0,0.0,0.0,True
107,107,2023-08-19,Sat,Away,Brentford,Fulham,Aaron-Hickey,1.0,RB,77.0,...,1.02,1775.117318,1767.636702,NaN,1.413793,1.0,0.0,0.0,0.0,True
108,108,2023-08-26,Sat,Home,Brentford,Crystal Palace,Aaron-Hickey,1.0,RB,90.0,...,0.69,1774.125598,1769.467049,NaN,1.466667,1.0,0.0,0.0,0.0,True


In [22]:
X_test.head(3)

,Unnamed: 0,Date,Day,Venue,Team,Opponent,Name,Start,Pos,Min,...,FPL_xGC_30,Opp_rating_30,Team_rating_30,Min_points_30,Team_result_30,DEF,FWD,GK,MID,Finished
76,76,2023-08-20,Sun,Home,West Ham,Chelsea,Aaron-Cresswell,0.0,0,0.0,...,0.00,1777.369108,1761.774133,NaN,1.166667,1.0,0.0,0.0,0.0,True
77,77,2023-08-26,Sat,Away,West Ham,Brighton,Aaron-Cresswell,0.0,0,0.0,...,0.00,1780.288375,1762.710238,NaN,1.166667,1.0,0.0,0.0,0.0,True
106,106,2023-08-13,Sun,Home,Brentford,Tottenham,Aaron-Hickey,1.0,RB,71.0,...,1.02,1773.661743,1765.611925,NaN,1.428571,1.0,0.0,0.0,0.0,True


# Predicting

In [23]:
def adjust_for_injuries(df):
    active_players = pd.read_csv("data/misc/active_players.csv")
    injured_players = active_players[active_players["chance_of_playing_this_round"] == 0]["name"].to_list()
    df.loc[df["Name_original"].isin(injured_players),"Pred"] = 0
    return df

In [24]:
def adjust_goalkeepers(df):
    gks = pd.read_csv("data/misc/goalkeepers.csv")
    gk_names = gks["Name"].to_list()
    gk_names = [name_fbref_to_fpl(neutralize_name(n)) for n in gk_names]
    # print(gk_names)
    df.loc[(df["FPL_pos"] == "GK") & ~(df["Name_original"].apply(neutralize_name).isin(gk_names)), "Pred"] = 0
    return df

In [25]:
def get_predictions(model, df, X):
    # make predictions on the test data and glues them to the rest of the dataframe
    predictions = model.predict(X)
    
    df_predictions = df[(df["Season"] == CURRENT_SEASON)].reset_index(drop=True)
        
    print(len(predictions), df_predictions.shape)
        
    df_predictions.loc[:, "Pred"] = predictions
    preds = df_predictions[info + ["Pred"]]
    
    preds = adjust_goalkeepers(preds)
    preds = adjust_for_injuries(preds).sort_values(by=["Pred"], ascending = False)
    
    return preds

In [26]:
X_test[FEATURES_OUTFIELD].isna()

,Was_home,Rating_difference,Price,Transfers_balance,Avg_FPL_points,Min_2,Gls_2,Ast_2,xG_2,xA_2,...,Team_CS_30,Team_score_30,Opp_score_30,Team_result_30,Player_GC_30,FPL_points_30,DEF,FWD,GK,MID
76,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
77,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
106,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
107,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
108,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34509,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
34510,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
34511,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
34512,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [27]:
# X_test.shape,  model.predict(X_test).shape

In [28]:
%%time
preds = get_predictions(model, df, X_test)

16744 (16744, 274)
CPU times: total: 312 ms
Wall time: 207 ms


C:\users\user\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [29]:
# next gameweek
preds[preds["GW"] == NEXT_GAMEWEEK].head(50)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Pred
16082,Virgil van Dijk,4.0,2023-24,Liverpool,Aston Villa,1.0,1946.521606,1825.741455,DEF,6.0,6.604484
13502,Raheem Sterling,4.0,2023-24,Chelsea,Nott'ham Forest,1.0,1781.043579,1677.877930,MID,7.1,6.513856
1428,Alexis Mac Allister,4.0,2023-24,Liverpool,Aston Villa,1.0,1946.521606,1825.741455,MID,6.0,6.326555
1883,Andrew Robertson,4.0,2023-24,Liverpool,Aston Villa,1.0,1946.521606,1825.741455,DEF,6.5,6.310435
5514,Erling Haaland,4.0,2023-24,Manchester City,Fulham,1.0,2083.702148,1732.639404,FWD,14.0,5.805550
15802,Trent Alexander-Arnold,4.0,2023-24,Liverpool,Aston Villa,1.0,1946.521606,1825.741455,DEF,7.9,5.418963
12105,Mohamed Salah,4.0,2023-24,Liverpool,Aston Villa,1.0,1946.521606,1825.741455,MID,12.5,5.406872
6141,Gabriel dos Santos Magalhães,4.0,2023-24,Arsenal,Manchester Utd,1.0,1927.032104,1865.755249,DEF,4.9,5.378679
3314,Carlton Morris,4.0,2023-24,Luton,West Ham,1.0,1602.477417,1787.825562,FWD,5.5,5.233385
16606,Yoane Wissa,4.0,2023-24,Brentford,Bournemouth,1.0,1835.573242,1660.290894,FWD,6.2,5.181244


In [30]:
preds[(preds["FPL_pos"] == "GK") & (preds["GW"] == NEXT_GAMEWEEK)].head(30)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Pred
13989,Robert Sánchez,4.0,2023-24,Chelsea,Nott'ham Forest,1.0,1781.043579,1677.877930,GK,4.5,4.905106
1498,Alisson Ramses Becker,4.0,2023-24,Liverpool,Aston Villa,1.0,1946.521606,1825.741455,GK,5.5,4.518524
7819,Jason Steele,4.0,2023-24,Brighton,Newcastle Utd,1.0,1843.758179,1884.512817,GK,4.5,4.145588
14337,Sam Johnstone,4.0,2023-24,Crystal Palace,Wolves,1.0,1758.725586,1704.537109,GK,4.5,3.901677
12628,Nick Pope,4.0,2023-24,Newcastle Utd,Brighton,0.0,1884.512817,1843.758179,GK,5.5,3.710454
1008,Aaron Ramsdale,4.0,2023-24,Arsenal,Manchester Utd,1.0,1927.032104,1865.755249,GK,5.0,3.521737
8763,Jordan Pickford,4.0,2023-24,Everton,Sheffield United,0.0,1691.463989,1631.764282,GK,4.5,3.119220
11686,Matt Turner,4.0,2023-24,Nott'ham Forest,Chelsea,0.0,1677.877930,1781.043579,GK,4.0,3.041767
5026,Ederson Santana de Moraes,4.0,2023-24,Manchester City,Fulham,1.0,2083.702148,1732.639404,GK,5.5,2.897502
16291,Wes Foderingham,4.0,2023-24,Sheffield United,Everton,1.0,1631.764282,1691.463989,GK,4.5,2.812669


In [31]:
# all preds
preds.head(20)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Pred
315,Guglielmo Vicario,2.0,2023-24,Tottenham,Manchester Utd,1.0,1817.656738,1877.037720,GK,5.0,6.685888
16104,Virgil van Dijk,26.0,2023-24,Liverpool,Luton,1.0,1946.521606,1602.477417,DEF,6.0,6.604484
16087,Virgil van Dijk,9.0,2023-24,Liverpool,Everton,1.0,1946.521606,1691.463989,DEF,6.0,6.604484
16084,Virgil van Dijk,6.0,2023-24,Liverpool,West Ham,1.0,1946.521606,1787.825562,DEF,6.0,6.604484
16090,Virgil van Dijk,12.0,2023-24,Liverpool,Brentford,1.0,1946.521606,1835.573242,DEF,6.0,6.604484
16116,Virgil van Dijk,38.0,2023-24,Liverpool,Wolves,1.0,1946.521606,1704.537109,DEF,6.0,6.604484
16082,Virgil van Dijk,4.0,2023-24,Liverpool,Aston Villa,1.0,1946.521606,1825.741455,DEF,6.0,6.604484
16114,Virgil van Dijk,36.0,2023-24,Liverpool,Tottenham,1.0,1946.521606,1829.239014,DEF,6.0,6.604484
16109,Virgil van Dijk,31.0,2023-24,Liverpool,Sheffield United,1.0,1946.521606,1631.764282,DEF,6.0,6.604484
16108,Virgil van Dijk,30.0,2023-24,Liverpool,Brighton,1.0,1946.521606,1843.758179,DEF,6.0,6.604484


In [32]:
preds[preds["FPL_pos"] == "GK"].head(20)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Pred
315,Guglielmo Vicario,2.0,2023-24,Tottenham,Manchester Utd,1.0,1817.656738,1877.037720,GK,5.0,6.685888
314,Guglielmo Vicario,1.0,2023-24,Tottenham,Brentford,0.0,1815.873413,1824.330444,GK,5.0,5.790391
35,Alisson Ramses Becker,1.0,2023-24,Liverpool,Chelsea,0.0,1946.899780,1788.094360,GK,5.5,5.409084
8962,José Malheiro de Sá,29.0,2023-24,Wolves,Bournemouth,1.0,1704.537109,1660.290894,GK,5.0,5.380183
8949,José Malheiro de Sá,16.0,2023-24,Wolves,Nott'ham Forest,1.0,1704.537109,1677.877930,GK,5.0,5.380183
8968,José Malheiro de Sá,35.0,2023-24,Wolves,Luton,1.0,1704.537109,1602.477417,GK,5.0,5.380183
8953,José Malheiro de Sá,20.0,2023-24,Wolves,Everton,1.0,1704.537109,1691.463989,GK,5.0,5.380183
8959,José Malheiro de Sá,26.0,2023-24,Wolves,Sheffield United,1.0,1704.537109,1631.764282,GK,5.0,5.380183
12746,Norberto Murara Neto,17.0,2023-24,Bournemouth,Luton,1.0,1660.290894,1602.477417,GK,4.5,5.266074
12757,Norberto Murara Neto,28.0,2023-24,Bournemouth,Sheffield United,1.0,1660.290894,1631.764282,GK,4.5,5.266074


In [33]:
# points all season
pd.DataFrame(preds.groupby(["Name_original"])["Pred"].sum().sort_values(ascending=False)).head(30)

,Pred
Name_original,
Virgil van Dijk,223.212429
Alexis Mac Allister,221.299522
Andrew Robertson,217.453839
Pervis Estupiñán,209.899412
Mohamed Salah,194.982986
Erling Haaland,191.767928
Callum Wilson,191.221981
Ollie Watkins,188.070351
Carlton Morris,185.659030


# Saving to file

In [34]:
preds.to_csv("predictions/preds_next_season.csv", index=False)